In [1]:
import plotly.graph_objects as go

from apollo.calculations.average_true_range import AverageTrueRangeCalculator
from apollo.calculations.elliot_waves_calculator import ElliotWavesCalculator
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, TICKER
from apollo.utils.common import to_default_date_string

In [2]:
start_date = "1995-01-01"
end_date = "2024-09-01"

window_size = 5

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

atr_calculator = AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
atr_calculator.calculate_average_true_range()

ew_calculator = ElliotWavesCalculator(
    dataframe=dataframe,
    window_size=window_size,
    fast_oscillator_period=5,
    slow_oscillator_period=35,
)
ew_calculator.calculate_elliot_waves()

dataframe = dataframe.loc["2007-06-01":"2008-03-01"]

dataframe

50 -1.0
53 -1.0
59 -1.0
62 -1.0
69 -1.0
81 -1.0
85 -1.0
88 -1.0
95 -1.0
107 -1.0
116 -1.0
119 -1.0
130 -1.0
133 -1.0
144 -1.0
169 -1.0
172 -1.0
175 -1.0
178 -1.0
181 -1.0
201 -1.0
210 1.0
213 -1.0
216 -1.0
219 -1.0
222 -1.0
225 -1.0
228 -1.0
231 -1.0
234 -1.0
237 -1.0
253 -1.0
260 1.0
263 1.0
270 -1.0
273 -1.0
276 -1.0
279 -1.0
282 -1.0
302 1.0
307 -1.0
310 -1.0
318 -1.0
321 1.0
324 1.0
330 -1.0
333 -1.0
336 -1.0
341 1.0
345 -1.0
348 -1.0
351 -1.0
354 -1.0
378 -1.0
381 -1.0
382 1.0
385 1.0
388 1.0
391 1.0
404 -1.0
407 -1.0
412 -1.0
415 -1.0
429 -1.0
432 -1.0
435 -1.0
440 -1.0
446 -1.0
449 -1.0
456 -1.0
468 -1.0
471 -1.0
474 -1.0
478 -1.0
481 -1.0
484 -1.0
496 1.0
501 -1.0
504 -1.0
513 -1.0
516 -1.0
519 -1.0
522 -1.0
530 -1.0
537 -1.0
540 -1.0
553 -1.0
559 1.0
562 1.0
567 1.0
570 1.0
587 -1.0
590 -1.0
593 -1.0
596 -1.0
599 -1.0
618 -1.0
621 -1.0
624 -1.0
635 -1.0
638 -1.0
643 -1.0
650 -1.0
653 -1.0
656 -1.0
663 1.0
666 1.0
672 1.0
675 1.0
680 -1.0
681 1.0
684 1.0
687 -1.0
690 -1.0
698 -

,ticker,adj close,adj high,adj low,adj open,adj volume,close,high,low,open,...,spf open,spf close,prev_close,tr,atr,high_low_avg,fast_hla_sma,slow_hla_sma,ewo,ewt
date,,,,,,,,,,,,,,,,,,,,,
2007-06-01,SPY,110.408325,110.637620,109.999878,110.265014,7.722542e+07,154.080002,154.399994,153.509995,153.880005,...,1534.00,1539.500000,109.863731,0.773889,0.901658,110.318749,109.415880,107.593354,1.822526,0.0
2007-06-04,SPY,110.422668,110.630467,109.978391,110.021383,5.589837e+07,154.100006,154.389999,153.479996,153.539993,...,1539.25,1540.250000,110.408325,0.652077,0.851742,110.304429,109.750513,107.779558,1.970955,0.0
2007-06-05,SPY,109.985565,110.279349,109.534126,110.164706,9.094492e+07,153.490005,153.899994,152.860001,153.740005,...,1539.75,1533.750000,110.422668,0.888543,0.859102,109.906737,109.951871,107.923690,2.028181,1.0
2007-06-06,SPY,108.803253,109.598641,108.738765,109.534153,1.175861e+08,151.839996,152.949997,151.750000,152.860001,...,1533.50,1516.500000,109.985565,1.246800,0.936642,109.168703,109.938981,108.032712,1.906270,-1.0
2007-06-07,SPY,106.839821,109.276137,106.811152,108.602564,1.665401e+08,149.100006,152.500000,149.059998,151.559998,...,1516.75,1489.250000,108.803253,2.464985,1.242310,108.043645,109.548453,108.107337,1.441115,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-02-25,SPY,99.825844,100.058448,97.972234,98.524678,1.381897e+08,137.330002,137.649994,134.779999,135.539993,...,1354.50,1371.599976,98.582832,2.086213,1.934678,99.015341,98.425113,98.995531,-0.570418,1.0
2008-02-26,SPY,100.574547,101.003418,99.222503,99.404230,1.544096e+08,138.360001,138.949997,136.500000,136.750000,...,1371.00,1382.750000,99.825844,1.780915,1.903926,100.112961,98.712233,98.903109,-0.190876,0.0
2008-02-27,SPY,100.472816,101.141568,99.884025,99.993056,1.224078e+08,138.220001,139.139999,137.410004,137.559998,...,1381.75,1380.500000,100.574547,1.257543,1.774649,100.512796,99.165822,98.843087,0.322735,1.0


In [3]:
value_1 = "adj close"
value_2 = "ewt"
value_3 = "adj high"
value_4 = "adj low"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]}"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        trace1,
        trace2,
        # trace3,
        # trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)